# Baseline CNN Model Notebook

In [18]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import collections
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
import pickle
import torch.utils.data
import torch.optim as optim
from model import ConvNet
import torchvision
import torchvision.transforms as transforms

### Pre-Processing the Data

In [3]:
def parseTrainData(topicLabels, df):
    data = {}
    labels = {}
    
    for dataType in ['train']:
        data[dataType] = {}
        labels[dataType] = {}
        
        for topic in topicLabels:
            data[dataType][topic] = []
            labels[dataType][topic] = []
            
            for idx, sentence in df.iterrows(): # iterates through each sentence in df we passed
                if sentence.category == topic:
                    data[dataType][topic].append(sentence.text)
                    labels[dataType][topic].append(topicLabels[sentence.category]) # assigns it the value in our dict of topicLabels based on each category
            
            assert len(data[dataType][topic]) == len(labels[dataType][topic]), \
                    "{}/{} data size does not match labels size".format(data_type, topic)
    return data, labels

At this stage, we have in read in the raw training and testing data and formatted it properly, and now we are going to shuffle the records

In [4]:
def prepareTrainData(data, labels):
    """Prepare training set from L3 Data"""
    
    # Combine pro and anti sentences and labels
    dataTrain = data['train']['pro-immigration'] + data['train']['anti-immigration'] + data['train']['pro-guns'] + data['train']['anti-guns'] + data['train']['pro-medicare'] + data['train']['oppose-medicare']
    labelsTrain = labels['train']['pro-immigration'] + labels['train']['anti-immigration'] + labels['train']['pro-guns'] + labels['train']['anti-guns'] + labels['train']['pro-medicare'] + labels['train']['oppose-medicare']
    
    # Shuffle sentences and the corresponding labels within the training data
    dataTrain, labelsTrain = shuffle(dataTrain, labelsTrain)
    
    # Return shuffled training data and training labels
    return dataTrain, labelsTrain

    

In [5]:
def prepareTestData(candidates):
    testDict = {}
    for candidate in candidates:
        df = pd.read_csv('./data/' + candidate + '_cleaned.csv')
        convertToList = df.text.to_list()
        wordsTest = [sentenceToWords(sentence) for sentence in convertToList]
        testDict[candidate] = wordsTest
    return testDict

Now we want to tokenize our input.

In [6]:
def sentenceToWords(sentence):
    nltk.download("stopwords", quiet = True)
    # stemmer = PorterStemmer()
    
    sentence = re.sub(r"[^a-zA-Z0-9]", " ", sentence.lower()) # Convert text to lower case
    words = sentence.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    # words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

Now we will apply the above method to all of our data and cache the results as well.

In [7]:
cacheDir = os.path.join("../cache", "cnn_analysis")  # where to store cache files
os.makedirs(cacheDir, exist_ok=True)  # ensure cache directory exists

def preprocessData(dataTrain, labelsTrain, cacheDir = cacheDir, cacheFile = "preprocessedData.pkl"):
    """Convert each review to words; read from cache if available."""
    
    # If cache_file is not None, try to read from it first
    cacheData = None
    if cacheFile is not None:
        try:
            pass
        except:
            pass  # unable to read from cache, but that's okay
        
      # If cache is missing, then do the heavy lifting
    if cacheData is None:
        # Preprocess training and test data to obtain words for each review
        # words_train = list(map(review_to_words, data_train))
        # words_test = list(map(review_to_words, data_test))
        wordsTrain = [sentenceToWords(sentence) for sentence in dataTrain]
        
        # Write to cache file for future runs
        if cacheFile is not None:
            cacheData = dict(wordsTrain=wordsTrain, labelsTrain=labelsTrain)
            
            with open(os.path.join(cacheDir, cacheFile), "wb") as f:
                pickle.dump(cacheData, f)
            print("Wrote preprocessed data to cache file:", cacheFile)
        
    else:
        # Unpack data loaded from cache file
        wordsTrain, labelsTrain = (cacheData['wordsTrain'], cacheData['labelsTrain'])
        
    return wordsTrain, labelsTrain


### Transform the Data

We will construct a feature representation that represents each word as an integer and include the words that appear most frequently.  We will combine all the infrequent words into another category by itself.  

In [8]:
def buildDict(data, vocabSize = 50000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    # Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    # sentence is a list of words.
    
    # A dict storing the words that appear in the reviews along with how often they occur
    wordCount = {}
    for sentence in data:
        for word in sentence:
            wordCount[word] = wordCount[word] + 1 if word in wordCount else 1
            
    # Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    # sorted_words[-1] is the least frequently appearing word. 
    wordCountSorted = sorted(wordCount.items(), key=(lambda item: item[1]), reverse=True)
    sortedWords = [item[0] for item in wordCountSorted]
    
    # This is what we are building, a dictionary that translates words into integers
    wordDict = {}
    for idx, word in enumerate(sortedWords[:vocabSize - 2]): # The -2 is so that we save room for the 'no word'
        wordDict[word] = idx + 2
    return wordDict

Now we have our word dictionary, so let's convert our sentences to integer sequence representation and pad our results to a fixed length.  

In [9]:
def convertAndPad(wordDict, sentence, pad = 500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1  # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    workingSentence = [NOWORD] * pad
    
    for wordIdx, word in enumerate(sentence[:pad]):
        if word in wordDict:
            workingSentence[wordIdx] = wordDict[word]
        else:
            workingSentence[wordIdx] = INFREQ
        
    return workingSentence, min(len(sentence), pad)

def convertAndPadData(wordDict, data, pad = 500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convertAndPad(wordDict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
    
    return np.array(result), np.array(lengths)

### Upload the Data

### Build and Train the PyTorch Model

In [10]:
def train(model, trainLoader, epochs, optimizer, criterion, device):
    total_step = len(trainLoader)
    for epoch in range(epochs):
        for idx, (data, labels) in enumerate(trainLoader):
            data = data.to(device)
            labels = data.to(device)

            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (id+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, epochs, id+1, total_step, loss.item()))


### Deploy the Model for Testing

### Use the Model for Testing

## Main Function for Calls

In [25]:
def main():
    # Initial Variables
    topicLabels = {
        'pro-immigration': 0,
        'anti-immigration': 1,
        'pro-guns': 2,
        'anti-guns': 3,
        'pro-medicare': 4,
        'oppose-medicare': 5
    }
    candidates = ['Biden', 'Buttigieg', 'Klobuchar', 'Sanders', 'Warren', 'Yang']
    Out.clear()
    
    data, labels = parseTrainData(topicLabels, df = pd.read_csv('./data/L3train_nonOneHot.csv'))
    
    # Let's check out the length of our training data for each category
    print("Immigration articles: train = {} pro / {} anti".format(
            len(data['train']['pro-immigration']), len(data['train']['anti-immigration'])))
    
    print("Immigration articles: train = {} pro / {} anti".format(
            len(data['train']['pro-guns']), len(data['train']['anti-guns'])))
    
    print("Immigration articles: train = {} pro / {} anti".format(
            len(data['train']['pro-medicare']), len(data['train']['oppose-medicare'])))
    
    # Below is our prepared shuffled training data
    trainX, trainY = prepareTrainData(data, labels)
    print(collections.Counter(trainY))
    # Below is our test set
    testX = prepareTestData(candidates)
    
    print("Sentences (combined): train = {}".format(len(trainX)))
    
    # Let's go check out an example from our training data now
    print(trainX[100])
    
    # Applying the tokenizer to get the stems
    print(sentenceToWords(trainX[1]))
    # Now we preprocess the Data
    trainX, trainY = preprocessData(trainX, trainY)
    # Build our word Dict
    wordDict = buildDict(trainX)
    print(list(wordDict.keys())[:5])
    
    # Save our wordDict
    dataDir = '../data/pytorch' # folder to store our data
    if not os.path.exists(dataDir): # check to make sure folder exists
        print("making dataDir folder...")
        os.makedirs(dataDir)
    
    with open(os.path.join(dataDir, 'wordDict.pkl'), 'wb') as f:
        pickle.dump(wordDict, f)
    
    # Now we pad all the sentences in our training data
    trainXNum, trainXLen = convertAndPadData(wordDict, trainX)
    
    # Pad all the sentences in our testing data?
    
    # Save our data locally
    pd.concat([pd.DataFrame(trainY), pd.DataFrame(trainXLen), pd.DataFrame(trainXNum)], axis=1) \
        .to_csv(os.path.join(dataDir, 'lstm_train.csv'), header=False, index=False)
    
    # Let's load a small portion of our training dataset for testing
    trainSample = pd.read_csv(os.path.join(dataDir, 'lstm_train.csv'), header=None, names=None, nrows=250)
    
    # Turn input df into tensors
    trainSampleY = torch.from_numpy(trainSample[[0]].values).float().squeeze()
    trainSampleX = torch.from_numpy(trainSample.drop([0], axis = 1).values).long()
    
    # Build the dataset
    trainSampleDS = torch.utils.data.TensorDataset(trainSampleX, trainSampleY)
    # Build the dataloader
    trainSampleDL = torch.utils.data.DataLoader(trainSampleDS, batch_size=50)
    
    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConvNet(10).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for bt in trainSampleDL:
        bX, bY = bt

        bX = bX.to(device)
        bY = bY.to(device)

        model = LSTMClassifier(100, 100, 50000).to(device)
        optimizer.zero_grad()
        output = model(bX)
        print(output)
        print(output.shape)
    # train(model, trainSampleDL, 5, optimizer, criterion, device)
if __name__ == "__main__":
    main()

Immigration articles: train = 900 pro / 385 anti
Immigration articles: train = 759 pro / 745 anti
Immigration articles: train = 673 pro / 639 anti
Counter({0: 900, 2: 759, 3: 745, 4: 673, 5: 639, 1: 385})
Sentences (combined): train = 4101
Suderman describes the inference he tries to draw from Shumlin’s cold feet as “the strongest argument against Mr. Sanders’s single-player plan.” As a defender of Sanders’s plan, I hope he’s right.
['fifty', 'four', 'years', 'ago', 'united', 'states', 'took', 'important', 'step', 'towards', 'universal', 'health', 'care', 'passing', 'medicare', 'program', 'law']
Wrote preprocessed data to cache file: preprocessedData.pkl
['gun', 'would', 'health', 'immigrants', 'medicare']
tensor([[0.1731, 0.1662, 0.1523, 0.1753, 0.1792, 0.1539],
        [0.1932, 0.1775, 0.1590, 0.1535, 0.1684, 0.1484],
        [0.1924, 0.1628, 0.1577, 0.1465, 0.1789, 0.1617],
        [0.1851, 0.1712, 0.1732, 0.1513, 0.1576, 0.1615],
        [0.2096, 0.1604, 0.1525, 0.1474, 0.1828, 0.1

tensor([[0.1672, 0.1549, 0.1608, 0.1660, 0.1850, 0.1661],
        [0.1720, 0.1794, 0.1494, 0.1643, 0.1669, 0.1681],
        [0.1473, 0.1668, 0.1659, 0.1732, 0.1700, 0.1767],
        [0.1710, 0.1586, 0.1534, 0.1528, 0.1875, 0.1768],
        [0.1688, 0.1566, 0.1433, 0.1761, 0.1786, 0.1766],
        [0.1666, 0.1701, 0.1558, 0.1789, 0.1810, 0.1475],
        [0.1765, 0.1732, 0.1556, 0.1475, 0.1750, 0.1723],
        [0.1537, 0.1676, 0.1771, 0.1608, 0.1660, 0.1748],
        [0.1731, 0.1690, 0.1662, 0.1652, 0.1655, 0.1609],
        [0.1740, 0.1687, 0.1662, 0.1444, 0.1742, 0.1725],
        [0.1630, 0.1589, 0.1524, 0.1649, 0.2079, 0.1529],
        [0.1611, 0.1920, 0.1620, 0.1554, 0.1562, 0.1733],
        [0.1641, 0.1646, 0.1883, 0.1577, 0.1610, 0.1642],
        [0.1809, 0.1691, 0.1471, 0.1672, 0.1731, 0.1627],
        [0.1798, 0.1624, 0.1432, 0.1646, 0.1612, 0.1888],
        [0.1805, 0.1514, 0.1736, 0.1564, 0.1843, 0.1539],
        [0.1870, 0.1484, 0.1833, 0.1752, 0.1722, 0.1339],
        [0.172

In [23]:
dataDir = '../data/pytorch' # folder to store our data
trainSample = pd.read_csv(os.path.join(dataDir, 'train.csv'), header=None, names=None, nrows=500)
trainSampleY = torch.from_numpy(trainSample[[0]].values).float().squeeze()
trainSampleX = torch.from_numpy(trainSample.drop([0], axis = 1).values).long()

# Build the dataset
trainSampleDS = torch.utils.data.TensorDataset(trainSampleX, trainSampleY)
# Build the dataloader
trainSampleDL = torch.utils.data.DataLoader(trainSampleDS, batch_size=50)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for bt in trainSampleDL:
    bX, bY = bt

    bX = bX.to(device)
    bY = bY.to(device)

    model = LSTMClassifier(100, 100, 50000).to(device)
    optimizer.zero_grad()
    output = model(bX)
    print(output)
    print(output.shape)

tensor([[0.1733, 0.1834, 0.1696, 0.1656, 0.1359, 0.1722],
        [0.1610, 0.1884, 0.1714, 0.1526, 0.1457, 0.1810],
        [0.1865, 0.1918, 0.1700, 0.1541, 0.1477, 0.1499],
        [0.1907, 0.1581, 0.1635, 0.1572, 0.1530, 0.1775],
        [0.1827, 0.1669, 0.1594, 0.1589, 0.1565, 0.1756],
        [0.1938, 0.1808, 0.1573, 0.1567, 0.1455, 0.1659],
        [0.1994, 0.1981, 0.1613, 0.1390, 0.1428, 0.1594],
        [0.1855, 0.2037, 0.1696, 0.1492, 0.1375, 0.1545],
        [0.1926, 0.1728, 0.1761, 0.1471, 0.1549, 0.1565],
        [0.1639, 0.1891, 0.1582, 0.1491, 0.1594, 0.1804],
        [0.1656, 0.1695, 0.1541, 0.1684, 0.1480, 0.1943],
        [0.1767, 0.1844, 0.1768, 0.1505, 0.1540, 0.1574],
        [0.1705, 0.1851, 0.1870, 0.1488, 0.1559, 0.1528],
        [0.1784, 0.1722, 0.1840, 0.1426, 0.1362, 0.1865],
        [0.1838, 0.1781, 0.1554, 0.1695, 0.1648, 0.1484],
        [0.1850, 0.1722, 0.1859, 0.1615, 0.1564, 0.1390],
        [0.1618, 0.1718, 0.1902, 0.1626, 0.1497, 0.1639],
        [0.191

tensor([[0.1721, 0.1628, 0.1590, 0.1578, 0.1613, 0.1870],
        [0.1797, 0.1455, 0.1505, 0.1557, 0.1778, 0.1908],
        [0.1727, 0.1632, 0.1708, 0.1450, 0.1705, 0.1779],
        [0.1608, 0.1524, 0.1471, 0.1719, 0.2007, 0.1671],
        [0.1884, 0.1405, 0.1391, 0.1570, 0.1941, 0.1809],
        [0.1905, 0.1534, 0.1528, 0.1702, 0.1531, 0.1799],
        [0.1909, 0.1662, 0.1473, 0.1711, 0.1486, 0.1759],
        [0.1757, 0.1564, 0.1711, 0.1598, 0.1556, 0.1814],
        [0.1465, 0.1577, 0.1629, 0.1845, 0.1711, 0.1774],
        [0.1744, 0.1643, 0.1738, 0.1566, 0.1590, 0.1719],
        [0.1704, 0.1940, 0.1569, 0.1482, 0.1570, 0.1736],
        [0.1886, 0.1433, 0.1690, 0.1660, 0.1699, 0.1632],
        [0.1682, 0.1653, 0.1634, 0.1521, 0.1751, 0.1759],
        [0.1707, 0.1653, 0.1477, 0.1465, 0.1815, 0.1884],
        [0.1547, 0.1547, 0.1756, 0.1728, 0.1816, 0.1606],
        [0.1916, 0.1647, 0.1457, 0.1612, 0.1469, 0.1898],
        [0.1598, 0.1590, 0.1641, 0.1872, 0.1754, 0.1545],
        [0.145

tensor([[0.1411, 0.1885, 0.1980, 0.1850, 0.1525, 0.1349],
        [0.1607, 0.1865, 0.1540, 0.1813, 0.1650, 0.1526],
        [0.1670, 0.1757, 0.1891, 0.1612, 0.1638, 0.1432],
        [0.1645, 0.1720, 0.1968, 0.1669, 0.1565, 0.1432],
        [0.1427, 0.1575, 0.1837, 0.1837, 0.1511, 0.1814],
        [0.1634, 0.1642, 0.1686, 0.1875, 0.1515, 0.1648],
        [0.1540, 0.1880, 0.1625, 0.1843, 0.1723, 0.1389],
        [0.1519, 0.1655, 0.1737, 0.1698, 0.1905, 0.1486],
        [0.1491, 0.1917, 0.1937, 0.1710, 0.1597, 0.1349],
        [0.1610, 0.1779, 0.1866, 0.1530, 0.1596, 0.1619],
        [0.1533, 0.1897, 0.1574, 0.1810, 0.1816, 0.1369],
        [0.1678, 0.1797, 0.1555, 0.1829, 0.1579, 0.1562],
        [0.1510, 0.1675, 0.1645, 0.1861, 0.1631, 0.1678],
        [0.1476, 0.1695, 0.1771, 0.1838, 0.1507, 0.1713],
        [0.1428, 0.1677, 0.1491, 0.1914, 0.1619, 0.1872],
        [0.1511, 0.1834, 0.1847, 0.1662, 0.1684, 0.1461],
        [0.1497, 0.1820, 0.1844, 0.1810, 0.1556, 0.1474],
        [0.161

tensor([[0.1675, 0.1677, 0.1683, 0.1647, 0.1731, 0.1586],
        [0.1658, 0.1642, 0.1669, 0.1528, 0.1827, 0.1676],
        [0.1868, 0.1388, 0.1645, 0.1561, 0.1883, 0.1655],
        [0.1850, 0.1652, 0.1503, 0.1627, 0.1887, 0.1481],
        [0.1884, 0.1419, 0.1480, 0.1715, 0.1819, 0.1682],
        [0.1717, 0.1496, 0.1590, 0.1719, 0.1730, 0.1747],
        [0.1620, 0.1588, 0.1646, 0.1659, 0.1919, 0.1568],
        [0.1890, 0.1516, 0.1731, 0.1488, 0.1826, 0.1549],
        [0.1954, 0.1307, 0.1747, 0.1573, 0.1898, 0.1520],
        [0.1967, 0.1481, 0.1554, 0.1586, 0.1708, 0.1705],
        [0.1889, 0.1727, 0.1492, 0.1692, 0.1742, 0.1458],
        [0.1714, 0.1552, 0.1869, 0.1420, 0.1712, 0.1733],
        [0.1754, 0.1494, 0.1563, 0.1770, 0.1840, 0.1579],
        [0.1675, 0.1509, 0.1756, 0.1800, 0.1866, 0.1395],
        [0.1810, 0.1537, 0.1698, 0.1727, 0.1656, 0.1572],
        [0.1735, 0.1285, 0.1804, 0.1739, 0.1801, 0.1636],
        [0.1798, 0.1640, 0.1727, 0.1643, 0.1720, 0.1472],
        [0.165

In [26]:
trainSample

,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,501
0,4,9,146,1633,2529,3,1634,63,494,833,...,0,0,0,0,0,0,0,0,0,0
1,2,11,117,17,237,2125,110,2,203,69,...,0,0,0,0,0,0,0,0,0,0
2,4,5,2530,2126,1459,624,2531,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,18,268,344,3,1460,8,127,51,199,...,0,0,0,0,0,0,0,0,0,0
4,0,25,1462,835,3206,1057,37,5,1463,37,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,2,847,1564,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,1,24,706,36,1105,2294,1740,1966,138,1245,...,0,0,0,0,0,0,0,0,0,0
497,1,12,2296,436,1246,18,1128,758,3527,4956,...,0,0,0,0,0,0,0,0,0,0
498,2,7,127,17,1741,1742,2,334,815,0,...,0,0,0,0,0,0,0,0,0,0


AttributeError: 'DataLoader' object has no attribute 'shape'

In [19]:
class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=6)
        self.sm = nn.Softmax(dim = 1)
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        text = x[1:,:]
        embeds = self.embedding(text)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sm(out.squeeze())